In [1]:
import tensorflow.keras as keras

In [2]:
'''
通过顺序API创建简单模型
'''
model1=keras.Sequential()
model1.add(keras.layers.Flatten(input_shape=[28,28]))  #将输入拉伸为一维数组
model1.add(keras.layers.Dense(300,activation="relu"))  #全连接层
model1.add(keras.layers.Dense(100,activation="relu"))
model1.add(keras.layers.Dense(10,activation="softmax")) #softmax多分类
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [3]:
'''
利用函数式API创建复杂网络
'''
input_A=keras.layers.Input(shape=(5),name="wide_input")#模型输入类型的规范
input_B=keras.layers.Input(shape=(6),name="deep_input")#多输入
hidden1=keras.layers.Dense(30,activation="relu")(input_A)#调用input_A作为输入
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat=keras.layers.Concatenate()([input_B,hidden1])#合并层
aux_output=keras.layers.Dense(1,name="aux_output")(hidden2)#多输出的 辅助输出层  (避免过拟合)
output=keras.layers.Dense(1,name="main_output")(concat)#输出层

model2=keras.Model(inputs=[input_A,input_B],outputs=[output,aux_output])#创建模型 指定输入和输出
model2.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 30)           180         wide_input[0][0]                 
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 36)           0           deep_input[0][0]                 
                                                                 dense_3[0][0]         

In [4]:
'''
子类API创建动态网络
'''
class WideandDeepModel(keras.Model):
    def __init__(self,units=30,activation="relu",**kwargs):
        super().__init__(**kwargs)
        self.hidden1=keras.layers.Dense(units,activation=activation)
        self.hidden2=keras.layers.Dense(units,activation=activation)
        self.main_output=keras.layers.Dense(1)
        self.aux_output=keras.layers.Dense(1)
        
    def call(self,inputs):
        input_A,input_B=inputs
        hidden1=self.hidden1(input_A)
        hidden2=self.hidden2(hidden1)
        concat=keras.layers.concatenate()([input_B,hidden2])
        main_output=self.main_output(concat)
        aux_output=self.aux_output(hidden2)
        return main_output,aux_output
    
model3=WideandDeepModel()
